<a href="https://colab.research.google.com/github/anatorres09/DE-MRI_Classificacio_CNN/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
# Clonar el repositori del GitHUb que conté les imatges
!git clone https://github.com/anatorres09/DE-MRI_Classificacio_CNN.git
! ls

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
fatal: destination path 'DE-MRI_Classificacio_CNN' already exists and is not an empty directory.
DE-MRI_Classificacio_CNN  drive  sample_data


In [2]:
# Connectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#Importar llibreries
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.gridspec import GridSpec

from skimage.filters import median, rank
from skimage.exposure import equalize_adapthist

from scipy import ndimage
from tensorflow import keras
from tensorflow.keras import layers
import nibabel as nib
import os
import urllib.request

import random
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from glob import glob

import shutil
from sklearn.preprocessing import LabelEncoder
import gc
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
data_path = '/content/DE-MRI_Classificacio_CNN/dataset' 
drive_folder = '/content/drive/MyDrive/CNN'


In [5]:
#CNN IMATGES .PNG

# Ruta de la carpeta principal
ruta_carpeta_normal = os.path.join(drive_folder, "corr_entrenament", "corr_normal")
ruta_carpeta_patologic = os.path.join(drive_folder, "corr_entrenament", "corr_patologic")


# Funció per recollir les imatges i les etiquetes
def recollir_imatges_etiquetes(ruta_carpeta, etiqueta):
    casos = [nom_carpeta for nom_carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, nom_carpeta))]

    imatges = []
    etiquetes = []

    for cas in casos:
        ruta_cas = os.path.join(ruta_carpeta, cas)
        rutes_imatges = glob(os.path.join(ruta_cas, '*.png'))

        for ruta_imatge in rutes_imatges:
            imatge = cv2.imread(ruta_imatge)
            imatge = cv2.cvtColor(imatge, cv2.COLOR_BGR2RGB)
            imatge = imatge.astype('float16') / 255.0  # Convertir a float16 y normalizar entre 0 y 1
            imatges.append(imatge)
            etiquetes.append(etiqueta)

    return imatges, etiquetes

# Recollir imatges i etiquetes de la carpeta corr_normal
imatges_normal, etiquetes_normal = recollir_imatges_etiquetes(ruta_carpeta_normal, "normal")

# Recollir imatges i etiquetes de la carpeta corr_patologic
imatges_patologic, etiquetes_patologic = recollir_imatges_etiquetes(ruta_carpeta_patologic, "patologic")

# Concatenar les llistes de imatges i etiquetes
imatges_total = imatges_normal + imatges_patologic
etiquetes_total = etiquetes_normal + etiquetes_patologic

# Convertir la llista de imatges i etiquetes en una matriu numpy
matriu_imatges = np.array(imatges_total, dtype='float16')
etiquetes = np.array(etiquetes_total)

print(f"S'han recollit {matriu_imatges.shape[0]} imatges en total.")


# Obtenir la quantitat de imatges etiquetades com "normal"
quantitat_normal = np.count_nonzero(etiquetes == "normal")

# Obtenir la quntitat de imatges etiquetades com "patologic"
quantitat_patologic = np.count_nonzero(etiquetes == "patologic")

print(f"S'han trobat {quantitat_normal} imatges etiquetades com 'normal'.")
print(f"S'han trobat {quantitat_patologic} imatges etiquetades com 'patologic'.")

S'han recollit 699 imatges en total.
S'han trobat 226 imatges etiquetades com 'normal'.
S'han trobat 473 imatges etiquetades com 'patologic'.


In [ ]:
# Preprocessat de les imatges
# Normalitzar els valors de píxels al rang [0, 1]
imatges = matriu_imatges/ 255.0

# Dividir les dades en conjunts d'entrenament i prova
X_train, X_test, y_train, y_test = train_test_split(imatges, etiquetes, test_size=0.2, random_state=42)



# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert encoded labels to one-hot encoding
y_train = tf.one_hot(y_train_encoded, depth=2)
y_test = tf.one_hot(y_test_encoded, depth=2)



# Construir el model de la CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compilar el model
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# Crear generadores de datos para el entrenamiento y la prueba
generador_entrenamiento = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3, dtype='float16')
generador_prueba = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3, dtype='float16')

# Crear generadores de flujo de datos para el entrenamiento y la prueba
flujo_entrenamiento = generador_entrenamiento.flow(X_train, y_train, batch_size=8)
flujo_prueba = generador_prueba.flow(X_test, y_test, batch_size=8)


# Antes de entrenar el modelo
gc.collect()  # Liberar la memoria antes del entrenamiento

# Entrenar el modelo utilizando generadores de datos
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

# Evaluar el modelo en el conjunto de prueba utilizando generador de datos
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Perdua en el conjunt de prova: {loss}")
print(f"Precisió en el conjunt de prova: {accuracy}")

# Liberar la memoria después de evaluar el modelo
del X_train, y_train, X_test, y_test
gc.collect()  # Liberar la memoria utilizada por las variables eliminadas


Epoch 1/10
70/70 [==============================] - ETA: 0s - loss: 0.6352 - accuracy: 0.6780